<a href="https://colab.research.google.com/github/parkrye/Python/blob/main/202210_Bigdata/03_RDD_Transformations_Actions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pyspark import SparkConf, SparkContext

In [ ]:
conf = SparkConf().setMaster("local").setAppName("transformations_actions")
sc = SparkContext(conf=conf)

스파크 환경 설정 확인

In [ ]:
sc.getConf().getAll()

[('spark.master', 'local'),
 ('spark.rdd.compress', 'True'),
 ('spark.app.id', 'local-1648807947231'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.submit.pyFiles', ''),
 ('spark.app.startTime', '1648807945887'),
 ('spark.executor.id', 'driver'),
 ('spark.driver.port', '51053'),
 ('spark.app.name', 'transformations_actions'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.driver.host', 'DESKTOP-F55CILR')]

# RDD 생성
일반 파이썬의 리스트를 이용해서 RDD 생성
- `parallelize([item1, item2, item3, ...])`

In [ ]:
foods = sc.parallelize([
    "짜장면", "마라탕", "짬뽕", "떡볶이", "쌀국수", "짬뽕", "짜장면", "짜장면", "짜장면", "라면", "우동", "라면"
])
foods

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [ ]:
foods.collect()

['짜장면', '마라탕', '짬뽕', '떡볶이', '쌀국수', '짬뽕', '짜장면', '짜장면', '짜장면', '라면', '우동', '라면']

각 음식 별 개수 세기
- `countByValue()`

In [ ]:
foods.countByValue()

defaultdict(int,
            {'짜장면': 4,
             '마라탕': 1,
             '짬뽕': 2,
             '떡볶이': 1,
             '쌀국수': 1,
             '라면': 2,
             '우동': 1})

상위 `n`개의 데이터 가져오기
- `take(n)`

In [ ]:
foods.take(10)

['짜장면', '마라탕', '짬뽕', '떡볶이', '쌀국수', '짬뽕', '짜장면', '짜장면', '짜장면', '라면']

In [ ]:
foods.take(3)

['짜장면', '마라탕', '짬뽕']

처음 1개의 데이터 가져오기
- `first()`

In [ ]:
foods.first()

'짜장면'

RDD 내 전체 데이터의 개수 세기
- `count()`

In [ ]:
foods.count()

12

중복 데이터 제거
- `distinct()`
- `transformation`

In [ ]:
fd = foods.distinct()
fd

PythonRDD[10] at RDD at PythonRDD.scala:53

In [ ]:
# 중복이 제거된 상태의 RDD
fd.collect()

['짜장면', '마라탕', '짬뽕', '떡볶이', '쌀국수', '라면', '우동']

In [ ]:
# 중복을 제외한 데이터 개수 세기
fd.count()

7

요소들을 하나 씩 꺼내서 함수에 저장
- `return`을 하지 않습니다.
- `foreach(<func>)`

In [ ]:
# worker 노드에서 실행이 됩니다. SparkContext에서 실행되는 것이 아닙니다.
foods.foreach(lambda x : print(x)) # x 에 RDD의 원소 하나하나 들어갑니다.

In [ ]:
foods.foreach(lambda x : x)

`foreach`는 `action`이기 때문에 `worker`노드에서 실행이 됩니다. 우리가 직접 실행하는 `Driver Program(SparkContext)`에서 코드가 실행되는 것이 아니기 때문에 SparkContext에서는 보이지 않습니다.

`foreach`는 RDD에 연산을 하고 나서 `log`를 저장하거나 할 때 유용합니다.

# Narrow Transformations
- 1:1 변환을 의미합니다.
- 하나의 열을 조작하기 위해 다른 열 및 파티션의 데이터를 사용할 필요가 없다.
- `filter()`, `map()`, `flatMap()`, `sample()`, `union()`

**`map(<func>)`**
- 데이터를 하나씩 꺼내서 `<func>` 함수가 적용된 새로운 RDD가 만들어 진다.

In [ ]:
sample_rdd = sc.parallelize([1, 2, 3])
sample_rdd

ParallelCollectionRDD[14] at readRDDFromFile at PythonRDD.scala:274

In [ ]:
sample_rdd2 = sample_rdd.map(lambda x : x + 2)
sample_rdd2

PythonRDD[15] at RDD at PythonRDD.scala:53

In [ ]:
sample_rdd2.collect()

[3, 4, 5]

**`flatMap(<func>)`**
- `map` 함수와 거의 비슷하나, `flatMap` 함수는 `map`의 모든 결과를 1차원 배열 형식으로 평평(`flat`)하게 나타낸다.

In [ ]:
movies = [
    "그린 북",
    "매트릭스",
    "토이 스토리",
    "캐스트 어웨이",
    "포드 V 페라리",
    "보헤미안 랩소디",
    "빽 투 더 퓨처",
    "반지의 제왕",
    "죽은 시인의 사회"
]

In [ ]:
moviesRDD = sc.parallelize(movies)
moviesRDD

ParallelCollectionRDD[16] at readRDDFromFile at PythonRDD.scala:274

In [ ]:
mapMovies = moviesRDD.map(lambda x : x.split(" "))
mapMovies

PythonRDD[17] at RDD at PythonRDD.scala:53

In [ ]:
mapMovies.collect()

[['그린', '북'],
 ['매트릭스'],
 ['토이', '스토리'],
 ['캐스트', '어웨이'],
 ['포드', 'V', '페라리'],
 ['보헤미안', '랩소디'],
 ['빽', '투', '더', '퓨처'],
 ['반지의', '제왕'],
 ['죽은', '시인의', '사회']]

In [ ]:
flatMovies = moviesRDD.flatMap(lambda x : x.split(" "))
flatMovies.collect()

['그린',
 '북',
 '매트릭스',
 '토이',
 '스토리',
 '캐스트',
 '어웨이',
 '포드',
 'V',
 '페라리',
 '보헤미안',
 '랩소디',
 '빽',
 '투',
 '더',
 '퓨처',
 '반지의',
 '제왕',
 '죽은',
 '시인의',
 '사회']

매트릭스만 제거한 RDD 생성

In [ ]:
filteredMovie = flatMovies.filter(lambda x : x != "매트릭스")
filteredMovie.collect()

['그린',
 '북',
 '토이',
 '스토리',
 '캐스트',
 '어웨이',
 '포드',
 'V',
 '페라리',
 '보헤미안',
 '랩소디',
 '빽',
 '투',
 '더',
 '퓨처',
 '반지의',
 '제왕',
 '죽은',
 '시인의',
 '사회']

**집합 `Transformation`**

In [ ]:
num1 = sc.parallelize([1, 2, 3, 4, 5])
num2 = sc.parallelize([4, 5, 6, 7, 8, 9, 10])

교집합 구하기 - `intersection`

In [ ]:
num1.intersection(num2).collect()

[4, 5]

합집합 구하기 - `union`

In [ ]:
num1.union(num2).collect()

[1, 2, 3, 4, 5, 4, 5, 6, 7, 8, 9, 10]

차집합 구하기 - `subtract`

In [ ]:
num1.subtract(num2).collect()

[2, 1, 3]

데이터 랜덤 추출 - `sample(withReplacement, fraction, seed=None)`
- 샘플링 : 데이터에서 일부분을 추출
- `withReplacement` : 한 번 추출 된 샘플을 다시 샘플링 대상으로 삼을 것인지 여부
    - `True` : 한 번 샘플링 된 데이터가 다시 대상이 됨
    - `False` : 한 번 샘플링 된 데이터는 다시 대상이 되지 않습니다.
- `fraction` : 샘플링 될 데이터의 기댓값(확률)
    - 각각의 데이터가 추출될 확률
    - 높아지면 높아질 수록 원본에서 샘플링되는 원소의 개수가 많아진다.

In [ ]:
numUnion = num1.union(num2)
numUnion.collect()

[1, 2, 3, 4, 5, 4, 5, 6, 7, 8, 9, 10]

In [ ]:
numUnion.sample(True, 0.5).collect()

[1, 2, 2, 3, 4, 4, 4]

In [ ]:
# withReplacement : True -> 중복 추출
numUnion.sample(True, 0.3).collect()

[5, 7, 7, 9, 10]

In [ ]:
# withReplacement : False -> 중복 X
numUnion.sample(False, 0.7).collect()

[1, 2, 4, 5, 5, 6, 7, 8, 9]

In [ ]:
# 랜덤을 고정해서 항상 같은 결과가 나올 수 있도록
numUnion.sample(True, 0.5, seed=42).collect()

[5, 6, 6]

# Wide Transformations
- `groupBy(<func>)`

In [ ]:
foods = sc.parallelize([
    "짜장면", "마라탕", "짬뽕", "떡볶이", "쌀국수", "짬뽕", "짜장면", "짜장면", "짜장면", "라면", "우동", "라면"
])
foods

ParallelCollectionRDD[55] at readRDDFromFile at PythonRDD.scala:274

In [ ]:
# 그룹핑의 기준을 문자열의 첫 번째 글자로 설정
foodsGroup = foods.groupBy(lambda x : x[0])
foodsGroup

PythonRDD[60] at RDD at PythonRDD.scala:53

In [ ]:
res = foodsGroup.collect()
res

[('짜', <pyspark.resultiterable.ResultIterable at 0x257b83228e0>),
 ('마', <pyspark.resultiterable.ResultIterable at 0x257b83220a0>),
 ('짬', <pyspark.resultiterable.ResultIterable at 0x257b8327c40>),
 ('떡', <pyspark.resultiterable.ResultIterable at 0x257b83279a0>),
 ('쌀', <pyspark.resultiterable.ResultIterable at 0x257b8327d00>),
 ('라', <pyspark.resultiterable.ResultIterable at 0x257b8327f10>),
 ('우', <pyspark.resultiterable.ResultIterable at 0x257b8327f70>)]

In [ ]:
for (k, v) in res:
    print(k, list(v))

짜 ['짜장면', '짜장면', '짜장면', '짜장면']
마 ['마라탕']
짬 ['짬뽕', '짬뽕']
떡 ['떡볶이']
쌀 ['쌀국수']
라 ['라면', '라면']
우 ['우동']


In [ ]:
sc.stop()